In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
!conda install -c conda-forge folium --yes

Solving environment: done

# All requested packages already installed.



In [3]:
import folium # map rendering library
from pandas import json_normalize # tranform JSON file into a pandas dataframe

In [4]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# List an array
postalCodeList = []
boroughList = []
neighborhoodList = []

In [7]:
# Locate the table and postal code
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [8]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip("\n"))
        boroughList.append(cells[1].text.rstrip("\n"))
        neighborhoodList.append(cells[2].text.rstrip("\n"))

In [9]:
# Load the pandas dataframe
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head(10)

PostalCode           Borough                                 Neighborhood
0        M1A      Not assigned                                 Not assigned
1        M2A      Not assigned                                 Not assigned
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
7        M8A      Not assigned                                 Not assigned
8        M9A         Etobicoke      Islington Avenue, Humber Valley Village
9        M1B       Scarborough                               Malvern, Rouge

In [10]:
# Remove no-assigned columns
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [11]:
# Group neighbour in the same borough
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [12]:
# Neighborhood WHICH IS "Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [13]:
# Notebook to build the code to scrape the following Wikipedia page, 
# https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to 
# to obtain the data that is in the table of postal codes and to transform the data into 
# into a pandas dataframe like the one shown below
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

PostalCode           Borough  \
0         M5G  Downtown Toronto   
1         M2H        North York   
2         M4B         East York   
3         M1J       Scarborough   
4         M4G         East York   
5         M4M      East Toronto   
6         M1R       Scarborough   
7         M9V         Etobicoke   
8         M9L        North York   
9         M5V  Downtown Toronto   
10        M1B       Scarborough   
11        M5A  Downtown Toronto   

                                         Neighborhood  
0                                  Central Bay Street  
1                                   Hillcrest Village  
2                     Parkview Hill, Woodbine Gardens  
3                                 Scarborough Village  
4                                             Leaside  
5                                     Studio District  
6                                   Wexford, Maryvale  
7   South Steeles, Silverstone, Humbergate, Jamest...  
8                                       Humber Summit  
9   CN Tower, King and Spadina, Railway Lands, Har...  
10                                     Malvern, Rouge  
11                          Regent Park, Harbourfront